# *Пташинская  Юлия*
> $Группа 80-304$
>
> $№ 12$


# $Logistic Regression$
Основная идея линейного классификатора заключается в том, что признаковое пространство может быть разделено гиперплоскостью на два полупространства, в каждом из которых прогнозируется одно из двух значений целевого класса.
Если это можно сделать без ошибок, то обучающая выборка называется линейно разделимой.
 ![texst](https://habrastorage.org/files/77b/af8/55c/77baf855c84a4ba9a851060ec7ae6ec8.png)

In [41]:
import pandas as pd
df = pd.read_csv('breast.csv')
df

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0
...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0
565,20.13,28.25,131.20,1261.0,0.09780,0
566,16.60,28.08,108.30,858.1,0.08455,0
567,20.60,29.33,140.10,1265.0,0.11780,0


In [42]:
#Загружаем базу данных 
import pandas as pd
df = pd.read_csv('breast.csv')
df = df.drop(df.columns[[-1]], axis=1)
df = df.to_numpy()
X = df
X
df = pd.read_csv('breast.csv')

df = df['diagnosis']
df = df.to_numpy()
y = df
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

### Link Function 
sigmoid - функция соединения. Она преобразует входное значение в диапазоне [0,1]. В моём случае всё, что меньше 0,5 - class 0, если больше 0,5 - class 1.

In [43]:
def sigmoid(score):
    return (1 / (1 + np.exp(-score)))

def predict_probability(features, weights):
    score = np.dot(features, weights)
    return sigmoid(score)

### Likelihood
С помощью вероятности можно определить насколько хорошо классификатор соответствует обучающим данным.
В качестве метрики используем Логарифмическая функция правдоподобия.
И градиентное восхождение для обновления веса.

In [44]:
def compute_log_likelihood(features, label, weights):
    indicator = (label==+1)
    scores = np.dot(features, weights)
    ll = np.sum((np.transpose(np.array([indicator]))-1)*scores - np.log(1. + np.exp(-scores)))
    return ll

In [45]:
# Производная 
def feature_derivative(errors, feature):
    derivative = np.dot(np.transpose(errors), feature)
    return derivative

In [46]:
# Логистическая регрессия без регуляризации(не направляем веса)
import matplotlib.pyplot as plt
def logistic_regression(features, labels, lr, epochs):   
    bias = np.ones((features.shape[0], 1))
    features = np.hstack((bias, features))

  # задаём веса
    weights = np.zeros((features.shape[1], 1)) #инцилизация вектора весов(случайные значения или ноль)
    logs = []

    for epoch in range(epochs):
        predictions = predict_probability(features, weights) # вычисляем вероятность для всех обучающих примеров
        indicators = (labels==+1) # вычисление значений индекатора(сравнивая с +1)
        errors = np.transpose(np.array([indicators])) - predictions # вычисляем ошибку 
       
    # для каждого весового коэффициента вычисляем его производную, lr - скорость обучения
        for j in range(len(weights)):
            derivative = feature_derivative(errors, features[:,j]) 
            weights[j] += lr * derivative

        ll = compute_log_likelihood(features, labels, weights)
        logs.append(ll)
    
    return weights

Для проверки производительности классификатора, разделю данные на обучающие и тестовые.

In [47]:
#Разбиваем dataset на обучающий и тестовый
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=9)

print("X_train : " + str(X_train.shape))
print("y_train : " + str(y_train.shape))
print("X_test : " + str(X_test.shape))
print("y_test : " + str(y_test.shape))

X_train : (455, 5)
y_train : (455,)
X_test : (114, 5)
y_test : (114,)


In [49]:
import numpy as np
# гипер параметры 
learning_rate = 1e-7
epochs = 100000

# логистическая регрессия
learned_weights = logistic_regression(X_train, y_train, learning_rate, epochs)

In [50]:
from sklearn.metrics import accuracy_score
# predictions, считаем accuracy
bias_train = np.ones((X_train.shape[0], 1))
bias_test = np.ones((X_test.shape[0], 1))
features_train = np.hstack((bias_train, X_train))
features_test  = np.hstack((bias_test, X_test))

test_predictions  = (predict_probability(features_test, learned_weights).flatten()>0.5)
train_predictions = (predict_probability(features_train, learned_weights).flatten()>0.5)
print("Accuracy на обущающей выборке: {}".format(accuracy_score(np.expand_dims(y_train, axis=1), train_predictions)))
print("Accuracy на тестовой выборке: {}".format(accuracy_score(np.expand_dims(y_test, axis=1), test_predictions)))


Accuracy на обущающей выборке: 0.8791208791208791
Accuracy на тестовой выборке: 0.8859649122807017
